<a href="https://colab.research.google.com/github/SeemGoel/Multimodal-Phi2/blob/experiments/V4_Training_eval_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Cap_project

/content/drive/MyDrive/Cap_project


In [3]:
# !pip install -q -U flash-attn #--no-build-isolation

In [4]:
# !pip install -q -U bitsandbytes
# !pip install -Uq -q accelerate peft bitsandbytes trl dataset bitsandbytes
# !pip install -q -U accelerate

In [5]:
# !pip -q install datasets

In [6]:
# !pip -q install -r requirements.txt

In [7]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [8]:
#Import necessary modules
import traceback #Import the traceback module
import os # Import the os module

In [9]:
import os
import datasets
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [10]:
# !wget -c -q "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"

In [11]:
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import torch
from torch.utils.data import DataLoader, RandomSampler
from config import get_training_args
from transformers import AutoModelForCausalLM
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, DatasetDict
import os
import json
from datetime import datetime
import torch
from transformers import AutoTokenizer
import traceback

from transformers import TrainerCallback, TrainerState, TrainerControl
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback # Import EarlyStoppingCallback

import torch
import gc
from typing import Dict, Union
import os

In [12]:
from torch.utils.checkpoint import checkpoint_sequential
#from modal import ProjectionBlock,MultimodalProjector,Phi3WithProjector
from modal import ProjectionBlock, EnhancedMultimodalProjector,Phi3WithProjector
from features import load_embeddings,transcribe_speech, MultimodalDataCollator, prepare_dataset, prepare_example

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
from config import get_training_args

In [14]:
from transformers import TrainerCallback
from tqdm.auto import tqdm
from datetime import datetime
import os
import torch

class ProgressCallback(TrainerCallback):
    def __init__(self, early_stopping_threshold=0.05):
        super().__init__()
        self.training_bar = None
        self.epoch_bar = None
        self.best_loss = float('inf')
        self.loss_history = []

    def on_train_begin(self, args, state, control, **kwargs):
        self.epoch_bar = tqdm(total=args.num_train_epochs, desc="Epochs", position=0)

    def on_step_end(self, args, state, control, **kwargs):
        if self.training_bar is None:
            self.training_bar = tqdm(total=state.max_steps, desc="Training", position=1)
        self.training_bar.update(1)

        if state.log_history:
            latest_logs = state.log_history[-1]
            loss = latest_logs.get('loss', None)
            if loss:
                self.loss_history.append(loss)
                recent_avg_loss = sum(self.loss_history[-100:]) / len(self.loss_history[-100:])
                self.training_bar.set_description(
                    f"Training - Loss: {loss:.4f} | Avg Loss (100): {recent_avg_loss:.4f}"
                )

                # Check for loss stability
                if len(self.loss_history) >= 100:
                    std_dev = torch.tensor(self.loss_history[-100:]).std().item()
                    if std_dev < self.early_stopping_threshold:
                        print(f"\nLoss has stabilized (std_dev: {std_dev:.4f})")

    def on_epoch_end(self, args, state, control, **kwargs):
        if self.epoch_bar:
            self.epoch_bar.update(1)
            if self.loss_history:
                avg_loss = sum(self.loss_history[-100:]) / len(self.loss_history[-100:])
                print(f"\nEpoch {state.epoch}: Average Loss = {avg_loss:.4f}")

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            eval_loss = metrics.get('eval_loss', None)
            if eval_loss:
                print(f"\nEvaluation Loss: {eval_loss:.4f}")
                if eval_loss < self.best_loss:
                    self.best_loss = eval_loss
                    print(f"New best evaluation loss!")

    def on_train_end(self, args, state, control, **kwargs):
        if self.training_bar:
            self.training_bar.close()
        if self.epoch_bar:
            self.epoch_bar.close()

In [15]:
def compute_metrics(eval_pred):
    """Compute metrics for validation"""
    predictions, labels = eval_pred

    # Calculate loss
    loss = torch.nn.functional.cross_entropy(
        torch.tensor(predictions).view(-1, predictions.shape[-1]),
        torch.tensor(labels).view(-1)
    )

    # Calculate accuracy (ignoring padding tokens)
    pred_ids = predictions.argmax(-1)
    valid_mask = labels != -100  # Ignore padding tokens
    accuracy = (pred_ids[valid_mask] == labels[valid_mask]).mean()

    # Calculate perplexity
    perplexity = torch.exp(loss)

    return {
        "loss": loss.item(),
        "accuracy": accuracy.item(),
        "perplexity": perplexity.item()
    }

class ValidationProgressCallback(TrainerCallback):
    """Callback to track and display validation metrics"""
    def __init__(self):
        self.best_accuracy = 0
        self.best_loss = float('inf')
        self.history = []

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            current_step = state.global_step
            current_accuracy = metrics.get("eval_accuracy", 0)
            current_loss = metrics.get("eval_loss", float('inf'))
            current_perplexity = metrics.get("eval_perplexity", float('inf'))

            # Update best scores
            if current_accuracy > self.best_accuracy:
                self.best_accuracy = current_accuracy
            if current_loss < self.best_loss:
                self.best_loss = current_loss

            # Store history
            self.history.append({
                "step": current_step,
                "accuracy": current_accuracy,
                "loss": current_loss,
                "perplexity": current_perplexity
            })

            # Print progress
            print(f"\nValidation Metrics at step {current_step}:")
            print(f"├─ Current Accuracy: {current_accuracy:.4f} (Best: {self.best_accuracy:.4f})")
            print(f"├─ Current Loss: {current_loss:.4f} (Best: {self.best_loss:.4f})")
            print(f"└─ Perplexity: {current_perplexity:.4f}")

            # Plot progress if matplotlib is available
            try:
                import matplotlib.pyplot as plt

                # Create progress plots
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

                steps = [h["step"] for h in self.history]
                accuracies = [h["accuracy"] for h in self.history]
                losses = [h["loss"] for h in self.history]

                # Accuracy plot
                ax1.plot(steps, accuracies, 'b-', label='Accuracy')
                ax1.set_title('Validation Accuracy Progress')
                ax1.set_xlabel('Steps')
                ax1.set_ylabel('Accuracy')
                ax1.grid(True)

                # Loss plot
                ax2.plot(steps, losses, 'r-', label='Loss')
                ax2.set_title('Validation Loss Progress')
                ax2.set_xlabel('Steps')
                ax2.set_ylabel('Loss')
                ax2.grid(True)

                plt.tight_layout()

                # Save plot
                plot_path = os.path.join(args.output_dir, f'validation_progress_{current_step}.png')
                plt.savefig(plot_path)
                plt.close()

                print(f"\nProgress plot saved to: {plot_path}")

            except ImportError:
                print("\nMatplotlib not available for plotting progress")



In [16]:
from features import prepare_dataset, MultimodalDataCollator
from modal import Phi3WithProjector, MultimodalProjector
from config import get_training_args
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from transformers import EarlyStoppingCallback
from tqdm.auto import tqdm

In [17]:
def continue_existing_finetuning(
    model_path ="/content/drive/MyDrive/LLM_training_20November/final_model",
    dataset_dir="/content/drive/MyDrive/Cap_dataset",
    output_dir="/content/drive/MyDrive/LLM_training_continued_v2",
    checkpoint_dir=None,
    max_train_samples=10000,  # Added parameter for dataset size control
    max_eval_samples=1000
):
    """Continue training an already fine-tuned model"""

    import gc
    import torch
    from datetime import datetime

    # Create timestamped output directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    output_dir = f"{output_dir}_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)

    # Memory cleanup before starting
    gc.collect()
    torch.cuda.empty_cache()

    print(f"\nStarting continued fine-tuning at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Output directory: {output_dir}")

    from datetime import datetime
    import logging
    from tqdm.auto import tqdm

    # Setup logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f'{output_dir}/training_log.txt'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    logger.info(f"\nStarting continued fine-tuning at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    logger.info(f"Model path: {model_path}")
    logger.info(f"Dataset path: {dataset_dir}")
    logger.info(f"Output directory: {output_dir}")

    class DetailedProgressCallback(ProgressCallback):
        def __init__(self, early_stopping_threshold=0.05):
            super().__init__(early_stopping_threshold)
            self.training_bar = None
            self.epoch_bar = None
            self.best_loss = float('inf')
            self.loss_history = []
            self.eval_history = []

        def on_log(self, args, state, control, logs=None, **kwargs):
            if logs:
                logger.info(f"Step {state.global_step}: {logs}")
                if 'loss' in logs:
                    self.loss_history.append(logs['loss'])
                if 'eval_loss' in logs:
                    self.eval_history.append(logs['eval_loss'])
                    logger.info(f"Evaluation - Loss: {logs['eval_loss']:.4f}")

        def on_epoch_end(self, args, state, control, **kwargs):
            if self.epoch_bar:
                self.epoch_bar.update(1)
                if self.loss_history:
                    avg_loss = sum(self.loss_history[-100:]) / len(self.loss_history[-100:])
                    logger.info(f"Epoch {state.epoch}: Average Loss = {avg_loss:.4f}")

    """Continue training using existing components"""

    print(f"\nStarting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # 1. Load dataset from disk
    print("\nLoading dataset...")
    try:
        dataset_dict = DatasetDict.load_from_disk(dataset_dir)
        tokenizer = AutoTokenizer.from_pretrained(
            "microsoft/Phi-3-mini-128k-instruct",
            trust_remote_code=True
        )
        print("Dataset loaded successfully!")
        # Load and reduce dataset size
        print(f"Original dataset sizes:")
        print(f"Train size: {len(dataset_dict['train'])}")
        print(f"Validation size: {len(dataset_dict['validation'])}")

        # Reduce dataset sizes
        if max_train_samples and max_train_samples < len(dataset_dict['train']):
            dataset_dict['train'] = dataset_dict['train'].select(range(max_train_samples))

        if max_eval_samples and max_eval_samples < len(dataset_dict['validation']):
            dataset_dict['validation'] = dataset_dict['validation'].select(range(max_eval_samples))

        print(f"\nReduced dataset sizes:")
        print(f"Train size: {len(dataset_dict['train'])}")
        print(f"Validation size: {len(dataset_dict['validation'])}")

    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

    # 2. Load base model with quantization
    print("\nLoading model...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # 3. Create multimodal projector
    print("Setting up multimodal projector...")
    projector = MultimodalProjector(
        image_input_dim=512,  # Adjust based on your image embedding size
        audio_input_dim=512,  # Adjust based on your audio embedding size
        output_dim=base_model.config.hidden_size
    )

    # 4. Create combined model
    model = Phi3WithProjector(base_model.config, base_model, projector)

    # 5. Prepare for training
    print("Preparing model for training...")
    model = prepare_model_for_kbit_training(model)

    # 6. Add LoRA adapters
    print("Adding LoRA adapters...")
    # Setup LoRA configuration with matching rank
    lora_config = LoraConfig(
        r=8,  # Match the rank from checkpoint (8 instead of 16)
        lora_alpha=32,
        target_modules=[
            "self_attn.qkv_proj",
            "self_attn.o_proj",
        ],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        modules_to_save=["projector"]
    )

    model = get_peft_model(model, lora_config)



     # 6. Get training arguments - Create the actual TrainingArguments object
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,  # Reduced from 3/5 to 1
        per_device_train_batch_size=4,  # Increased from 4
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=16,  # Reduced from 8
        evaluation_strategy="steps",
        eval_steps=100,  # Less frequent evaluation
        save_strategy="steps",
        save_steps=100,  # Less frequent saving
        logging_steps=50,
        learning_rate=1e-4,  # Increased for faster convergence
        weight_decay=0.01,
        warmup_ratio=0.05,  # Reduced warmup
        lr_scheduler_type="cosine",  # Simpler scheduler
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        report_to="tensorboard",
        remove_unused_columns=False,
        optim="adamw_torch_fused",
        max_grad_norm=1.0,
    )
    # Custom callback for memory tracking and emergency saving
    class MemoryTracker(TrainerCallback):
        def __init__(self, warning_threshold=0.90):
            self.warning_threshold = warning_threshold
            self.last_save_step = 0
            self.min_steps_between_saves = 20

        def on_step_end(self, args, state, control, **kwargs):
            if torch.cuda.is_available():
                memory_used = torch.cuda.memory_allocated() / torch.cuda.max_memory_allocated()
                if memory_used > self.warning_threshold:
                    print(f"\nWarning: High memory usage detected ({memory_used:.2%})")
                    if (state.global_step - self.last_save_step) >= self.min_steps_between_saves:
                        # Force emergency save
                        emergency_dir = os.path.join(args.output_dir, f"emergency-save-{state.global_step}")
                        trainer.save_model(emergency_dir)
                        print(f"\nEmergency checkpoint saved at: {emergency_dir}")
                        self.last_save_step = state.global_step
                        # Cleanup
                        gc.collect()
                        torch.cuda.empty_cache()

    # Add explicit save points
    class SaveVerificationCallback(TrainerCallback):
        def on_save(self, args, state, control, **kwargs):
            checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")

            # Ensure model is saved
            try:
                trainer.save_model(checkpoint_dir)

                # Verify files exist
                required_files = ["adapter_config.json", "adapter_model.bin"]
                missing_files = [f for f in required_files
                               if not os.path.exists(os.path.join(checkpoint_dir, f))]

                if missing_files:
                    print(f"\nWarning: Missing files in checkpoint {state.global_step}: {missing_files}")
                    # Retry save if files are missing
                    print("Retrying save...")
                    trainer.save_model(checkpoint_dir)
                else:
                    print(f"\nCheckpoint {state.global_step} saved and verified successfully")

            except Exception as e:
                print(f"\nError saving checkpoint {state.global_step}: {str(e)}")
                raise


    # 7. Initialize trainer with enhanced monitoring
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['validation'],
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=2),
            DetailedProgressCallback(early_stopping_threshold=0.05),
            MemoryTracker(warning_threshold=0.90),
            ValidationProgressCallback(),
            SaveVerificationCallback()
        ],
        data_collator=MultimodalDataCollator(tokenizer),
        compute_metrics=compute_metrics
    )

    # 8. Start training with enhanced logging
    try:
        logger.info("Starting training...")
        result = trainer.train()

        # Log final metrics
        logger.info("\nTraining completed!")
        logger.info(f"Total training time: {result.metrics['train_runtime']:.2f} seconds")
        logger.info(f"Training samples per second: {result.metrics['train_samples_per_second']:.2f}")
        logger.info(f"Final loss: {result.metrics['train_loss']:.4f}")

        # Save the final model
        final_model_path = os.path.join(output_dir, "final_model")
        trainer.save_model(final_model_path)
        logger.info(f"Model saved to {final_model_path}")

        # Save final model with verification
        final_model_path = os.path.join(output_dir, "final_model")
        trainer.save_model(final_model_path)

        # Verify saved model
        verification_files = ["adapter_config.json", "adapter_model.bin"]
        missing_files = [f for f in verification_files
                        if not os.path.exists(os.path.join(final_model_path, f))]

        if missing_files:
            print(f"\nWarning: Some model files are missing: {missing_files}")
        else:
            print(f"\nModel successfully saved to {final_model_path}")

        return result, final_model_path

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        # Try emergency save
        try:
            emergency_path = os.path.join(output_dir, "emergency_final_save")
            trainer.save_model(emergency_path)
            print(f"\nEmergency save completed at: {emergency_path}")
        except:
            print("\nFailed to complete emergency save")
        raise

    finally:
        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        logger.info(f"\nFine-tuning finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
if __name__ == "__main__":
    try:
        # Continue fine-tuning from your existing model
        results = continue_existing_finetuning(
            model_path="/content/drive/MyDrive/LLM_training_continued_v2_20241123_1723/final_model",
            checkpoint_dir="/content/drive/MyDrive/LLM_training_output_1Nov2024_V3/checkpoint-904",
            dataset_dir="/content/drive/MyDrive/Cap_dataset",
            output_dir="/content/drive/MyDrive/LLM_training_continued_v2"
        )

    except Exception as e:
        print(f"\nFine-tuning failed: {str(e)}")
        traceback.print_exc()


Starting continued fine-tuning at 2024-11-23 18:46:29
Output directory: /content/drive/MyDrive/LLM_training_continued_v2_20241123_1846

Starting training at 2024-11-23 18:46:29

Loading dataset...
Dataset loaded successfully!
Original dataset sizes:
Train size: 289128
Validation size: 36141

Reduced dataset sizes:
Train size: 10000
Validation size: 1000

Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from /content/drive/MyDrive/LLM_training_continued_v2_20241123_1723/final_model led to unexpected keys not found in the model:  ['phi3_model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'phi3_model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'phi3_model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'phi3_model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'phi3_model.model.layers.0.self_attn.qkv_proj.lora_A.default.weight', 'phi3_model.model.layers.0.self_attn.qkv_proj.lora_B.default.weight', 'phi3_model.model.layers.1.mlp.down_proj.lora_A.default.weight', 'phi3_model.model.layers.1.mlp.down_proj.lora_B.default.weight', 'phi3_model.model.layers.1.self_attn.o_proj.lora_A.default.weight', 'phi3_model.model.layers.1.self_attn.o_proj.lora_B.default.weight', 'phi3_model.model.layers.1.self_attn.qkv_proj.lora_A.default.weight', 'phi3_model.model.layers.1.self_attn.qkv_proj.lora_B.default.weight', 'phi3_model.model.layers.10.mlp.dow

Setting up multimodal projector...
Preparing model for training...
Adding LoRA adapters...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training:   0%|          | 0/156 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss


In [34]:

results

(TrainOutput(global_step=11, training_loss=8.122658469460227, metrics={'train_runtime': 339.9826, 'train_samples_per_second': 0.98, 'train_steps_per_second': 0.032, 'total_flos': 2023161735118848.0, 'train_loss': 8.122658469460227, 'epoch': 0.0352}),
 '/content/drive/MyDrive/LLM_training_continued_v2_20241123_1735/final_model')

In [ ]:
def find_latest_model(base_path="/content/drive/MyDrive/"):
    """Find the latest properly saved model in Google Drive"""
    import glob
    import os
    from datetime import datetime

    print("Searching for saved models...")

    # Common patterns for model folders
    patterns = [
        "**/final_model",
        "**/LLM_training_*/final_model",
        "**/checkpoint-*"
    ]

    valid_models = []

    for pattern in patterns:
        search_path = os.path.join(base_path, pattern)
        folders = glob.glob(search_path, recursive=True)

        for folder in folders:
            # Check for essential files that indicate a complete model save
            if os.path.exists(os.path.join(folder, "adapter_config.json")) and \
               os.path.exists(os.path.join(folder, "adapter_model.bin")):

                # Get folder modification time
                mod_time = datetime.fromtimestamp(os.path.getmtime(folder))
                valid_models.append((mod_time, folder))

    if valid_models:
        # Sort by modification time, newest first
        valid_models.sort(reverse=True)
        print("\nFound valid models (newest first):")
        for i, (mod_time, path) in enumerate(valid_models):
            print(f"{i+1}. {path}")
            print(f"   Last modified: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")

        return valid_models[0][1]  # Return path of newest model
    else:
        print("No valid saved models found!")
        return None

# Usage:
latest_model = find_latest_model()
if latest_model:
    print(f"\nLatest valid model found at: {latest_model}")
    # Use this path in continue_existing_finetuning
    results = continue_existing_finetuning(
        model_path=latest_model,
        output_dir="/content/drive/MyDrive/LLM_training_continued_v2"
    )

Searching for saved models...


In [24]:
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import torch
from torch.utils.data import DataLoader, RandomSampler
from config import get_training_args
from transformers import AutoModelForCausalLM
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training


from transformers import TrainerCallback, TrainerState, TrainerControl
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback # Import EarlyStoppingCallback

import torch
import gc
from typing import Dict, Union
import os

# class OOMSafeCallback(TrainerCallback):
#     """Custom callback to handle OOM errors and ensure safe model saving"""
#     def __init__(self, save_dir: str):
#         self.save_dir = save_dir
#         self.best_loss = float('inf')

#     def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
#         """Check memory usage after each step"""
#         if torch.cuda.is_available():
#             # Monitor GPU memory
#             gpu_memory = torch.cuda.memory_allocated() / 1024**2  # Convert to MB
#             if gpu_memory > 0.95 * torch.cuda.get_device_properties(0).total_memory / 1024**2:
#                 print(f"Warning: High GPU memory usage detected: {gpu_memory:.2f}MB")

#     def on_save(self, args, state: TrainerState, control: TrainerControl, **kwargs):
#         """Handle safe model saving"""
#         try:
#             # Clear memory before saving
#             gc.collect()
#             torch.cuda.empty_cache()

#             # Get model from kwargs
#             model = kwargs.get('model')
#             if model is None:
#                 return

#             # Move model to CPU for saving if needed
#             device = next(model.parameters()).device
#             if device.type != 'cpu':
#                 model.to('cpu')

#             # Save in try-except block
#             try:
#                 checkpoint_dir = os.path.join(
#                     self.save_dir,
#                     f"checkpoint-{state.global_step}"
#                 )
#                 os.makedirs(checkpoint_dir, exist_ok=True)

#                 # Save model state separately
#                 model.save_pretrained(checkpoint_dir)

#                 # Save training state
#                 state_dict = {
#                     "step": state.global_step,
#                     "epoch": state.epoch,
#                     "best_loss": self.best_loss
#                 }
#                 torch.save(state_dict, os.path.join(checkpoint_dir, "trainer_state.bin"))

#             finally:
#                 # Move model back to original device
#                 if device.type != 'cpu':
#                     model.to(device)

#         except Exception as e:
#             print(f"Warning: Error during saving: {str(e)}")
#             # Attempt emergency save
#             try:
#                 emergency_save_path = os.path.join(self.save_dir, "emergency_save")
#                 os.makedirs(emergency_save_path, exist_ok=True)

#                 # Save in 8-bit format to reduce memory
#                 model.save_pretrained(
#                     emergency_save_path,
#                     save_in_8bit=True,
#                     max_shard_size="500MB"
#                 )
#                 print(f"Emergency save successful at: {emergency_save_path}")
#             except Exception as e2:
#                 print(f"Emergency save failed: {str(e2)}")


# def compute_metrics(eval_pred):
#     """Compute BLEU score for model evaluation"""
#     predictions = eval_pred.predictions
#     labels = eval_pred.label_ids

#     # Ensure predictions and labels are decoded to text first
#     # This depends on your tokenizer and data format
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     bleu_scores = []
#     for pred, label in zip(decoded_preds, decoded_labels):
#         # Convert strings to tokens for BLEU calculation
#         pred_tokens = pred.split()
#         label_tokens = label.split()
#         bleu_scores.append(sentence_bleu([label_tokens], pred_tokens))

#     avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
#     return {"bleu": avg_bleu}


# def train_model(dataset_dict, tokenizer, output_dir):
#     from transformers import BitsAndBytesConfig
#     from transformers import logging as transformers_logging

#     config = TrainingConfig()

#     # Set logging verbosity
#     transformers_logging.set_verbosity_info()  # Or set_verbosity_debug() for even more detailed output


#     # Configure quantization commented on 20 november
#     # quantization_config = BitsAndBytesConfig(
#     #     load_in_8bit=True,
#     #     load_in_8bit_fp32_cpu_offload=True,
#     #     llm_int8_threshold=6.0,
#     #     llm_int8_has_fp16_weight=False,
#     #     bnb_4bit_compute_dtype=torch.float16
#     # )
#     quantization_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_storage="float32",
#         bnb_4bit_compute_dtype="bfloat16",
#         bnb_4bit_use_double_quant=True
#     )
#     # Explicitly define device_map to load the model onto the GPU
#     device_map = {"": 0}  # Load the model to the first GPU (index 0

#     # Load the main model with dynamic quantization configuration
#     # phi3_model = AutoModelForCausalLM.from_pretrained(
#     #     "microsoft/Phi-3-mini-128k-instruct",
#     #     quantization_config=quantization_config,
#     #     torch_dtype=torch.float16,
#     #     device_map=device_map
#     # )

#     # Clear GPU memory before loading model
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#         gc.collect()

#     try:
#         phi3_model = AutoModelForCausalLM.from_pretrained(
#             "microsoft/Phi-3-mini-128k-instruct",
#             quantization_config=quantization_config,
#             torch_dtype=torch.float16,
#             device_map=device_map  # Let transformers handle device mapping
#         )
#     except Exception as e:
#         print(f"Error loading model: {str(e)}")
#         raise

#     phi3_model = prepare_model_for_kbit_training(phi3_model)


#     # Set output_dim dynamically based on the model's hidden size
#     output_dim = phi3_model.config.hidden_size

#     # Initialize MultimodalProjector with the dynamically set output_dim
#     projector = EnhancedMultimodalProjector(
#         image_input_dim=512,
#         audio_input_dim=512,
#         output_dim=output_dim    # Dynamic output dimension based on model's hidden size
#     )

#     # Initialize Phi3WithProjector model
#     model = Phi3WithProjector(phi3_model.config, phi3_model, projector )

#     # Configure LoRA commented to solve the problem of features as out put in prompt
#     # lora_config = LoraConfig(
#     #     r=8,
#     #     lora_alpha=16,
#     #     target_modules=["self_attn.q_proj",
#     #     "self_attn.k_proj",
#     #     "self_attn.v_proj",
#     #     "self_attn.o_proj",
#     #     "mlp.gate_proj",
#     #     "mlp.down_proj",
#     #     "mlp.up_proj"  ],
#     #     lora_dropout=0.05,
#     #     bias="none",
#     #     # save_embedding_layers=True,
#     #     task_type=TaskType.CAUSAL_LM
#     # )

#     # Configure LoRA Updated on 17th Nov 2024
#     lora_config = LoraConfig(
#         r=8, # updaeted from 8 to 16
#         lora_alpha=16, # updaeted from 16 to 32
#         target_modules=["self_attn.q_proj",
#         "self_attn.k_proj",
#         "self_attn.v_proj",
#         "self_attn.o_proj",
#         "mlp.gate_proj",
#         "mlp.down_proj",
#         "mlp.up_proj"  ],
#         lora_dropout=0.1,  # updaeted from .05 to .1
#         bias="none",
#         # save_embedding_layers=True,
#         task_type=TaskType.CAUSAL_LM
#     )



#     lora_config.save_embedding_layers = True
#     model = get_peft_model(model, lora_config)
#     num_samples_per_epoch = 500
#     sampler = RandomSampler(dataset_dict['train'],
#                             replacement=True,
#                             num_samples=num_samples_per_epoch
#                             ) # Use replacement for smaller datasets


#     # Add the OOMSafeCallback to the trainer's callbacks
#     callbacks = [
#         EarlyStoppingCallback(early_stopping_patience=3),
#         OOMSafeCallback(save_dir=output_dir)
#     ]




#     # Training arguments optimised
#     training_args = TrainingArguments(
#         output_dir=config.output_dir,

#       #@num_train_epochs=config.num_train_epochs,
#         num_train_epochs=1/5,
#         per_device_train_batch_size=config.per_device_train_batch_size,
#         gradient_accumulation_steps=config.gradient_accumulation_steps,
#         learning_rate=config.learning_rate,
#         #fp16=config.fp16,
#         bf16=True,
#         warmup_ratio=config.warmup_ratio,
#         lr_scheduler_type=config.lr_scheduler_type,
#         save_total_limit=config.save_total_limit,
#         logging_steps=config.logging_steps,
#         # max_grad_norm=config.max_grad_norm,
#         report_to=config.report_to,
#         save_steps=config.save_steps,
#         evaluation_strategy=config.evaluation_strategy,
#         eval_steps=config.eval_steps,
#         # logging_dir=config.logging_dir,
#         # logging_strategy=config.logging_strategy,
#         disable_tqdm=False,
#         eval_strategy=config.eval_strategy,
#         load_best_model_at_end=config.load_best_model_at_end,
#         save_strategy=config.save_strategy,
#         push_to_hub=config.push_to_hub,
#         # run_name=config.run_name,
#         # logging_first_step=config.logging_first_step,
#         remove_unused_columns=config.remove_unused_columns,
#         optim="adamw_torch",#config.optim,
#         gradient_checkpointing=True,#config.gradient_checkpointing,
#         gradient_checkpointing_kwargs={"use_reentrant": False},
#         resume_from_checkpoint=config.resume_from_checkpoint,

#          dataloader_pin_memory=False,  # Reduce CPU memory usage
#         # torch_compile=True,  # Enable torch.compile
#         # deepspeed=config.deepspeed,
#         # seed=config.seed,

#     )
#     # Create DataLoader with the sampler
#     train_dataloader = DataLoader(
#         dataset_dict['train'],
#         batch_size=training_args.per_device_train_batch_size,
#         sampler=sampler,
#         collate_fn=MultimodalDataCollator(tokenizer),
#         num_workers=1,  # Consider adjusting for optimal performance
#     )


#     # Trainer setup
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=dataset_dict['train'],
#         eval_dataset=dataset_dict['validation'],
#         processing_class=tokenizer,
#         compute_metrics=compute_metrics,
#         data_collator=MultimodalDataCollator(tokenizer),
#         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
#     )

#     train_result = None
#     # Wrap the training in a try-except block
#     try:
#         if os.path.exists(os.path.join(output_dir, PREFIX_CHECKPOINT_DIR)):
#             latest_checkpoint = max(
#                 [os.path.join(output_dir, d) for d in os.listdir(output_dir)
#                  if d.startswith(PREFIX_CHECKPOINT_DIR)],
#                 key=os.path.getmtime,
#             )
#             print(f"Loading model from checkpoint: {latest_checkpoint}")
#             model = Phi3WithProjector.from_pretrained(latest_checkpoint)
#         else:
#             train_result = trainer.train()

#     except RuntimeError as e:
#         if "out of memory" in str(e):
#             print("OOM error encountered. Attempting to save current state...")

#             # Clear memory
#             if torch.cuda.is_available():
#                 torch.cuda.empty_cache()
#             gc.collect()

#             # Save emergency checkpoint
#             emergency_dir = os.path.join(output_dir, "emergency_checkpoint")
#             trainer.save_model(emergency_dir)
#             print(f"Emergency checkpoint saved to {emergency_dir}")

#             # Optionally, you can exit here or continue with reduced batch size
#             raise e

#     # Save final model and metrics
#     try:
#         trainer.save_model(output_dir)
#         trainer.save_state()

#         if  train_result is not None:
#             metrics = train_result.metrics
#             trainer.log_metrics("train", metrics)
#             trainer.save_metrics("train", metrics)

#             eval_metrics = trainer.evaluate()
#             trainer.log_metrics("eval", eval_metrics)
#             trainer.save_metrics("eval", eval_metrics)
#     except Exception as e:
#         print(f"Error during final save: {str(e)}")
#         # Attempt one final emergency save
#         emergency_final_dir = os.path.join(output_dir, "final_emergency_save")
#         model.save_pretrained(emergency_final_dir, save_in_8bit=True)

#     return model, trainer


In [46]:
def get_lora_config():
    return LoraConfig(
        r=8,  # Rank dimension
        lora_alpha=16,
        target_modules = ["self_attn.q_proj",
                          "self_attn.k_proj",
                          "self_attn.v_proj",
                          "self_attn.o_proj", "mlp.gate_proj", "mlp.down_proj", "mlp.up_proj"  ],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        fan_in_fan_out=True,
        init_lora_weights="gaussian",
        modules_to_save=["projector"],  # Still save projector weights for multimodal components
    )

       # target_modules=[
        #     "self_attn.q_proj",  # Query projection
        #     "self_attn.k_proj",  # Key projection
        #     "self_attn.v_proj",  # Value projection
        # ],

In [60]:
class MultimodalDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        batch = {}

        # Handle text inputs
        input_ids = [f["input_ids"] for f in features]
        attention_mask = [f["attention_mask"] for f in features]
        labels = [f["labels"] for f in features]

        # Pad the sequences
        max_length = max(len(ids) for ids in input_ids)

        padded_input_ids = []
        padded_attention_mask = []
        padded_labels = []

        for ids, mask, lab in zip(input_ids, attention_mask, labels):
            padding_length = max_length - len(ids)

            padded_input_ids.append(ids + [self.tokenizer.pad_token_id] * padding_length)
            padded_attention_mask.append(mask + [0] * padding_length)
            padded_labels.append(lab + [-100] * padding_length)

        batch["input_ids"] = torch.tensor(padded_input_ids)
        batch["attention_mask"] = torch.tensor(padded_attention_mask)
        batch["labels"] = torch.tensor(padded_labels)

        # Handle image embeddings if present
        if "image_embeddings" in features[0]:
            image_embeddings = torch.stack([f["image_embeddings"] for f in features])
            # Store image embeddings separately to be handled by the model's forward pass
            batch["image_embeddings"] = image_embeddings

        # Handle audio embeddings if present
        if "audio_embeddings" in features[0]:
            audio_embeddings = torch.stack([f["audio_embeddings"] for f in features])
            batch["audio_embeddings"] = audio_embeddings

        return batch

class Phi3WithMultimodal(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, input_ids=None, attention_mask=None, labels=None,
                image_embeddings=None, audio_embeddings=None, **kwargs):
        # Process embeddings if present
        if image_embeddings is not None or audio_embeddings is not None:
            # Get the original embeddings
            inputs_embeds = self.base_model.get_input_embeddings()(input_ids)

            # Append image embeddings if present
            if image_embeddings is not None:
                image_proj = self.image_projection(image_embeddings)
                inputs_embeds = torch.cat([inputs_embeds, image_proj.unsqueeze(1)], dim=1)
                attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1),
                                        device=attention_mask.device)], dim=1)

            # Append audio embeddings if present
            if audio_embeddings is not None:
                audio_proj = self.audio_projection(audio_embeddings)
                inputs_embeds = torch.cat([inputs_embeds, audio_proj.unsqueeze(1)], dim=1)
                attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1),
                                        device=attention_mask.device)], dim=1)

            # Adjust labels if needed
            if labels is not None:
                labels = torch.cat([labels, torch.full((labels.shape[0], inputs_embeds.shape[1] - labels.shape[1]),
                                -100, device=labels.device)], dim=1)

            # Forward pass with embeddings
            outputs = self.base_model(inputs_embeds=inputs_embeds,
                                    attention_mask=attention_mask,
                                    labels=labels,
                                    **kwargs)
        else:
            # Standard forward pass
            outputs = self.base_model(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    labels=labels,
                                    **kwargs)

        return outputs



In [ ]:
def save_dataset(image_embeddings_path, dataset_path, save_dir="/content/drive/MyDrive/Cap_dataset"):
    """
    Prepare and save dataset to disk for future use
    """
    print(f"\nStarting dataset preparation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Check if dataset already exists
        if os.path.exists(save_dir):
            print(f"Dataset already exists at {save_dir}")
            user_input = input("Do you want to overwrite it? (yes/no): ")
            if user_input.lower() != 'yes':
                print("Keeping existing dataset")
                return

        # Prepare dataset
        print("\nPreparing dataset...")
        dataset_dict, tokenizer = prepare_dataset(
            image_embeddings_path=image_embeddings_path,
            dataset_path=dataset_path
        )

        # Create directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Save dataset
        print(f"\nSaving dataset to {save_dir}")
        dataset_dict.save_to_disk(save_dir)

        # Save some dataset statistics
        stats = {
            'train_size': len(dataset_dict['train']),
            'validation_size': len(dataset_dict['validation']),
            'test_size': len(dataset_dict['test']),
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'image_embeddings_path': image_embeddings_path,
            'dataset_path': dataset_path
        }

        # Save statistics
        with open(os.path.join(save_dir, 'dataset_info.json'), 'w') as f:
            json.dump(stats, f, indent=4)

        print("\nDataset saved successfully!")
        print(f"Train size: {stats['train_size']}")
        print(f"Validation size: {stats['validation_size']}")
        print(f"Test size: {stats['test_size']}")

    except Exception as e:
        print(f"\nError saving dataset: {str(e)}")
        import traceback
        traceback.print_exc()

    finally:
        print(f"\nProcess finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Usage example:
if __name__ == "__main__":
    # First prepare and save the dataset
    save_dataset(
        image_embeddings_path="/path/to/your/image_embeddings.npz",
        dataset_path="/path/to/your/dataset.json",
        save_dir="/content/drive/MyDrive/Cap_dataset"
    )


In [ ]:

    # Then you can run training using the saved dataset
    results = continue_training(
        model_path="microsoft/Phi-3-mini-128k-instruct",
        output_dir="/content/drive/MyDrive/LLM_training_continued"
    )

In [61]:
from transformers import BitsAndBytesConfig
from transformers import logging as transformers_logging
from tqdm.auto import tqdm
from transformers import TrainerCallback
from datetime import datetime

class ProgressCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.training_bar = None
        self.epoch_bar = None
        self.current_epoch = 0

    def on_train_begin(self, args, state, control, **kwargs):
        self.total_epochs = args.num_train_epochs
        self.epoch_bar = tqdm(total=self.total_epochs, desc="Epochs", position=0)

    def on_epoch_begin(self, args, state, control, **kwargs):
        self.current_epoch = state.epoch
        self.epoch_bar.set_description(f"Epoch {state.epoch:.2f}/{self.total_epochs}")
        self.epoch_bar.update(state.epoch - self.epoch_bar.n)

        if self.training_bar is None:
            total_steps = state.max_steps
            self.training_bar = tqdm(total=total_steps, desc="Training", position=1)

    def on_step_end(self, args, state, control, **kwargs):
        self.training_bar.update(1)

        # Update metrics in progress bar description
        if state.log_history:
            latest_logs = state.log_history[-1]
            loss = latest_logs.get('loss', None)
            lr = latest_logs.get('learning_rate', None)
            desc = f"Training - Loss: {loss:.4f}" if loss else "Training"
            if lr:
                desc += f", LR: {lr:.2e}"
            self.training_bar.set_description(desc)

    def on_epoch_end(self, args, state, control, **kwargs):
        self.epoch_bar.update(1)

    def on_train_end(self, args, state, control, **kwargs):
        self.epoch_bar.close()
        if self.training_bar is not None:
            self.training_bar.close()

def continue_training(model_path, train_dataset, eval_dataset, output_dir):
    """Continue training with progress bars"""

    print(f"\nStarting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Model path: {model_path}")
    print(f"Output directory: {output_dir}")

    # 1. Load model with quantization config
    print("\nLoading model...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # Wrap the model with multimodal support
    model = Phi3WithMultimodal(model)

    # Prepare for training
    model = prepare_model_for_kbit_training(model)

    # Add LoRA adapters
    lora_config = get_lora_config()
    model = get_peft_model(model, lora_config)

    # Print trainable parameters
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}%")

    # 4. Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=5e-5,
        fp16=True,
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine_with_restarts",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        gradient_checkpointing=True,
        report_to="tensorboard",
        remove_unused_columns=False,
    )

    # 5. Initialize trainer with progress callback
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=3),
            ProgressCallback()
        ],
        data_collator=MultimodalDataCollator(tokenizer)
    )

    print("\nStarting training...")
    try:
          # Check if there's a checkpoint
          checkpoint_dir = None
          if os.path.exists(output_dir):
              checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
              if checkpoints:
                  # Get the latest checkpoint
                  latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
                  checkpoint_dir = os.path.join(output_dir, latest_checkpoint)
                  print(f"Resuming from checkpoint: {checkpoint_dir}")
              else:
                  print("No checkpoint found, starting fresh training")
          else:
              print("Output directory doesn't exist, starting fresh training")
              os.makedirs(output_dir, exist_ok=True)

          result = trainer.train(resume_from_checkpoint=checkpoint_dir)

          print("\nTraining completed!")
          print(f"Total training time: {result.metrics['train_runtime']:.2f} seconds")
          print(f"Training samples per second: {result.metrics['train_samples_per_second']:.2f}")
          print(f"Final loss: {result.metrics['train_loss']:.4f}")

          return result

    except Exception as e:
            print(f"\nError during training: {str(e)}")
            raise

    finally:
            print(f"\nTraining finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
from features import prepare_dataset, MultimodalDataCollator
from modal import Phi3WithProjector, MultimodalProjector
from config import get_training_args
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from transformers import EarlyStoppingCallback
from tqdm.auto import tqdm

In [65]:


class ProgressCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.training_bar = None
        self.epoch_bar = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.epoch_bar = tqdm(total=args.num_train_epochs, desc="Epochs", position=0)

    def on_step_end(self, args, state, control, **kwargs):
        if self.training_bar is None:
            self.training_bar = tqdm(total=state.max_steps, desc="Training", position=1)
        self.training_bar.update(1)
        if state.log_history:
            latest_logs = state.log_history[-1]
            loss = latest_logs.get('loss', None)
            if loss:
                self.training_bar.set_description(f"Training - Loss: {loss:.4f}")

def continue_training(model_path, image_embeddings_path, dataset_path, output_dir):
    """Continue training using existing components"""

    print(f"\nStarting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # 1. Prepare dataset using existing function
    print("\nPreparing dataset...")
    dataset_dict, tokenizer = prepare_dataset(
        image_embeddings_path=image_embeddings_path,
        dataset_path=dataset_path
    )

    # 2. Load base model with quantization
    print("\nLoading model...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # 3. Create multimodal projector
    print("Setting up multimodal projector...")
    projector = MultimodalProjector(
        image_input_dim=1024,  # Adjust based on your image embedding size
        audio_input_dim=1024,  # Adjust based on your audio embedding size
        output_dim=base_model.config.hidden_size
    )

    # 4. Create combined model
    model = Phi3WithProjector(base_model.config, base_model, projector)

    # 5. Prepare for training
    print("Preparing model for training...")
    model = prepare_model_for_kbit_training(model)

    # 6. Add LoRA adapters
    print("Adding LoRA adapters...")
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=[
            "self_attn.qkv_proj",
            "self_attn.o_proj",
        ],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        modules_to_save=["projector"]
    )
    model = get_peft_model(model, lora_config)

    # 7. Get training arguments from config
    training_args = get_training_args(output_dir)

    # 8. Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['validation'],
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=3),
            ProgressCallback()
        ],
        data_collator=MultimodalDataCollator(tokenizer)
    )

    # 9. Start training
    print("\nStarting training...")
    try:
        result = trainer.train()
        print("\nTraining completed!")
        return result

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

    finally:
        print(f"\nTraining finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



In [68]:
def save_dataset(image_embeddings_path, dataset_path, save_dir="/content/drive/MyDrive/Cap_dataset"):
    """
    Prepare and save dataset to disk for future use
    """
    print(f"\nStarting dataset preparation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Check if dataset already exists
        if os.path.exists(save_dir):
            print(f"Dataset already exists at {save_dir}")
            user_input = input("Do you want to overwrite it? (yes/no): ")
            if user_input.lower() != 'yes':
                print("Keeping existing dataset")
                return

        # Prepare dataset
        print("\nPreparing dataset...")
        dataset_dict, tokenizer = prepare_dataset(
            image_embeddings_path=image_embeddings_path,
            dataset_path=dataset_path
        )

        # Create directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Save dataset
        print(f"\nSaving dataset to {save_dir}")
        dataset_dict.save_to_disk(save_dir)

        # Save some dataset statistics
        stats = {
            'train_size': len(dataset_dict['train']),
            'validation_size': len(dataset_dict['validation']),
            'test_size': len(dataset_dict['test']),
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'image_embeddings_path': image_embeddings_path,
            'dataset_path': dataset_path
        }

        # Save statistics
        with open(os.path.join(save_dir, 'dataset_info.json'), 'w') as f:
            json.dump(stats, f, indent=4)

        print("\nDataset saved successfully!")
        print(f"Train size: {stats['train_size']}")
        print(f"Validation size: {stats['validation_size']}")
        print(f"Test size: {stats['test_size']}")

    except Exception as e:
        print(f"\nError saving dataset: {str(e)}")
        import traceback
        traceback.print_exc()

    finally:
        print(f"\nProcess finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Usage example:
if __name__ == "__main__":
    # First prepare and save the dataset
    save_dataset(
       image_embeddings_path="/content/drive/MyDrive/LLM/Copy of clip_embeddings_instruct150k_15Oct_2024.npz",
        dataset_path="/content/drive/MyDrive/Cap_project/llava_instruct_150k.json",
        save_dir="/content/drive/MyDrive/Cap_dataset"
    )

    # # Then you can run training using the saved dataset
    # results = continue_training(
    #     model_path="microsoft/Phi-3-mini-128k-instruct",
    #     output_dir="/content/drive/MyDrive/LLM_training_continued"
    # )


Starting dataset preparation at 2024-11-23 15:32:22
Dataset already exists at /content/drive/MyDrive/Cap_dataset
Do you want to overwrite it? (yes/no): n
Keeping existing dataset

Process finished at 2024-11-23 15:32:55


In [73]:
from datasets import Dataset, DatasetDict
import os
import json
from datetime import datetime
import torch
from transformers import AutoTokenizer
import traceback

def verify_saved_dataset(dataset_dir="/content/drive/MyDrive/Cap_dataset"):
    """
    Verify that the dataset was saved correctly
    """
    try:
        print(f"\nVerifying dataset at {dataset_dir}")

        # Check if directory exists
        if not os.path.exists(dataset_dir):
            print(f"Dataset directory not found at {dataset_dir}")
            return False

        # Load dataset info
        info_path = os.path.join(dataset_dir, 'dataset_info.json')
        if os.path.exists(info_path):
            with open(info_path, 'r') as f:
                info = json.load(f)
                print("\nDataset Info:")
                print(f"Creation date: {info['creation_date']}")
                print(f"Expected train size: {info['train_size']}")
                print(f"Expected validation size: {info['validation_size']}")
                print(f"Expected test size: {info['test_size']}")

        # Try loading the dataset
        print("\nAttempting to load dataset...")
        dataset_dict = DatasetDict.load_from_disk(dataset_dir)

        # Verify sizes
        actual_sizes = {
            'train': len(dataset_dict['train']),
            'validation': len(dataset_dict['validation']),
            'test': len(dataset_dict['test'])
        }

        print("\nActual dataset sizes:")
        print(f"Train size: {actual_sizes['train']}")
        print(f"Validation size: {actual_sizes['validation']}")
        print(f"Test size: {actual_sizes['test']}")

        # Check a few samples
        print("\nChecking sample from training set:")
        sample = dataset_dict['train'][0]
        print(f"Sample keys: {list(sample.keys())}")

        # Verify data types
        print("\nVerifying data types:")
        for key in sample.keys():
            print(f"{key}: {type(sample[key])}")

        return True

    except Exception as e:
        print(f"\nError verifying dataset: {str(e)}")
        traceback.print_exc()
        return False

# For saving dataset
def save_dataset(image_embeddings_path, dataset_path, save_dir="/content/drive/MyDrive/Cap_dataset"):
    """
    Prepare and save dataset to disk for future use
    """
    from features import prepare_dataset  # Import here to avoid circular imports

    print(f"\nStarting dataset preparation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Check if dataset already exists
        if os.path.exists(save_dir):
            print(f"Dataset already exists at {save_dir}")
            user_input = input("Do you want to overwrite it? (yes/no): ")
            if user_input.lower() != 'yes':
                print("Keeping existing dataset")
                return

        # Prepare dataset
        print("\nPreparing dataset...")
        dataset_dict, tokenizer = prepare_dataset(
            image_embeddings_path=image_embeddings_path,
            dataset_path=dataset_path
        )

        # Create directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Save dataset
        print(f"\nSaving dataset to {save_dir}")
        dataset_dict.save_to_disk(save_dir)

        # Save some dataset statistics
        stats = {
            'train_size': len(dataset_dict['train']),
            'validation_size': len(dataset_dict['validation']),
            'test_size': len(dataset_dict['test']),
            'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'image_embeddings_path': image_embeddings_path,
            'dataset_path': dataset_path
        }

        # Save statistics
        with open(os.path.join(save_dir, 'dataset_info.json'), 'w') as f:
            json.dump(stats, f, indent=4)

        print("\nDataset saved successfully!")
        print(f"Train size: {stats['train_size']}")
        print(f"Validation size: {stats['validation_size']}")
        print(f"Test size: {stats['test_size']}")

    except Exception as e:
        print(f"\nError saving dataset: {str(e)}")
        traceback.print_exc()

    finally:
        print(f"\nProcess finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Usage example:
if __name__ == "__main__":
    # First verify if dataset exists
    if verify_saved_dataset():
        print("\nDataset verification successful!")
    else:
        print("\nDataset needs to be prepared...")
        # If verification fails, prepare and save the dataset
        save_dataset(
            image_embeddings_path = "/content/drive/MyDrive/LLM/Copy of clip_embeddings_instruct150k_15Oct_2024.npz",
            dataset_path = "/content/drive/MyDrive/Cap_project/llava_instruct_150k.json"
        )


Verifying dataset at /content/drive/MyDrive/Cap_dataset

Attempting to load dataset...

Actual dataset sizes:
Train size: 289128
Validation size: 36141
Test size: 36142

Checking sample from training set:
Sample keys: ['input_ids', 'attention_mask', 'labels', 'image_embeddings']

Verifying data types:
input_ids: <class 'torch.Tensor'>
attention_mask: <class 'torch.Tensor'>
labels: <class 'torch.Tensor'>
image_embeddings: <class 'torch.Tensor'>

Dataset verification successful!


In [75]:
from datasets import DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    EarlyStoppingCallback,
    BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from features import MultimodalDataCollator
from modal import Phi3WithProjector, MultimodalProjector
from config import get_training_args
import torch
from tqdm.auto import tqdm
import os
from datetime import datetime

class ProgressCallback(TrainerCallback):
    def __init__(self, early_stopping_threshold=0.1):
        super().__init__()
        self.training_bar = None
        self.epoch_bar = None
        self.best_loss = float('inf')
        self.loss_history = []

    def on_train_begin(self, args, state, control, **kwargs):
        self.epoch_bar = tqdm(total=args.num_train_epochs, desc="Epochs", position=0)

    def on_step_end(self, args, state, control, **kwargs):
        if self.training_bar is None:
            self.training_bar = tqdm(total=state.max_steps, desc="Training", position=1)
        self.training_bar.update(1)

        if state.log_history:
            latest_logs = state.log_history[-1]
            loss = latest_logs.get('loss', None)
            if loss:
                self.loss_history.append(loss)
                recent_avg_loss = sum(self.loss_history[-100:]) / len(self.loss_history[-100:])
                self.training_bar.set_description(
                    f"Training - Loss: {loss:.4f} | Avg Loss (100): {recent_avg_loss:.4f}"
                )

def continue_finetuning(
    model_path,
    dataset_dir="/content/drive/MyDrive/Cap_dataset",
    output_dir="/content/drive/MyDrive/LLM_training_continued",
    checkpoint_dir=None
):
    """Continue fine-tuning using saved dataset"""

    print(f"\nStarting fine-tuning at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # 1. Load dataset from disk
    print("\nLoading dataset...")
    try:
        dataset_dict = DatasetDict.load_from_disk(dataset_dir)
        tokenizer = AutoTokenizer.from_pretrained(
            "microsoft/Phi-3-mini-128k-instruct",
            trust_remote_code=True
        )
        print("Dataset loaded successfully!")
        print(f"Train size: {len(dataset_dict['train'])}")
        print(f"Validation size: {len(dataset_dict['validation'])}")

    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

    # 2. Load model with quantization
    print("\nLoading model...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # 3. Setup multimodal components
    print("Setting up multimodal projector...")
    projector = MultimodalProjector(
        image_input_dim=1024,  # Adjust based on your image embedding size
        audio_input_dim=1024,  # Adjust based on your audio embedding size
        output_dim=base_model.config.hidden_size
    )

    model = Phi3WithProjector(base_model.config, base_model, projector)

    # 4. Prepare for training
    print("Preparing model for training...")
    model = prepare_model_for_kbit_training(model)

    # 5. Add LoRA adapters
    print("Adding LoRA adapters...")
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=[
            "self_attn.qkv_proj",
            "self_attn.o_proj",
        ],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        modules_to_save=["projector"]
    )
    model = get_peft_model(model, lora_config)

    # 6. Get training arguments
    training_args = get_training_args(output_dir)

    # 7. Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['validation'],
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=3),
            ProgressCallback(early_stopping_threshold=0.1)
        ],
        data_collator=MultimodalDataCollator(tokenizer)
    )

    # 8. Start training
    print("\nStarting training...")
    try:
        # Check for existing checkpoint
        if checkpoint_dir is None and os.path.exists(output_dir):
            checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
            if checkpoints:
                latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
                checkpoint_dir = os.path.join(output_dir, latest_checkpoint)
                print(f"Found checkpoint: {checkpoint_dir}")

        result = trainer.train(resume_from_checkpoint=checkpoint_dir)

        # Print training summary
        print("\nTraining completed!")
        print(f"Total training time: {result.metrics['train_runtime']:.2f} seconds")
        print(f"Training samples per second: {result.metrics['train_samples_per_second']:.2f}")
        print(f"Final loss: {result.metrics['train_loss']:.4f}")

        # Save the final model
        final_model_path = os.path.join(output_dir, "final_model")
        trainer.save_model(final_model_path)
        print(f"Model saved to {final_model_path}")

        return result

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

    finally:
        print(f"\nFine-tuning finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Usage example:
if __name__ == "__main__":
    try:
        # First verify dataset exists
        if not os.path.exists("/content/drive/MyDrive/Cap_dataset"):
            print("Dataset not found! Please run save_dataset() first.")
        else:
            # Continue fine-tuning
            results = continue_finetuning(
                model_path="microsoft/Phi-3-mini-128k-instruct",
                dataset_dir="/content/drive/MyDrive/Cap_dataset",
                output_dir="/content/drive/MyDrive/LLM_training_continued"
            )

    except Exception as e:
        print(f"\nFine-tuning failed: {str(e)}")
        import traceback
        traceback.print_exc()


Starting fine-tuning at 2024-11-23 15:39:10

Loading dataset...
Dataset loaded successfully!
Train size: 289128
Validation size: 36141

Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting up multimodal projector...
Preparing model for training...
Adding LoRA adapters...

Fine-tuning failed: name 'TrainingArguments' is not defined


Traceback (most recent call last):
  File "<ipython-input-75-7696bba623bb>", line 176, in <cell line: 169>
    results = continue_finetuning(
  File "<ipython-input-75-7696bba623bb>", line 120, in continue_finetuning
    training_args = get_training_args(output_dir)
  File "/content/drive/MyDrive/Cap_project/config.py", line 2, in get_training_args
    AutoModelForCausalLM,
NameError: name 'TrainingArguments' is not defined


In [69]:
# Usage
if __name__ == "__main__":
    try:
        results = results = continue_training(
        model_path="microsoft/Phi-3-mini-128k-instruct",
        output_dir="/content/drive/MyDrive/LLM_training_continued"
    )

    except Exception as e:
        print(f"\nTraining failed: {str(e)}")
        import traceback
        traceback.print_exc()


Training failed: continue_training() missing 2 required positional arguments: 'image_embeddings_path' and 'dataset_path'


Traceback (most recent call last):
  File "<ipython-input-69-87502008d508>", line 4, in <cell line: 2>
    results = results = continue_training(
TypeError: continue_training() missing 2 required positional arguments: 'image_embeddings_path' and 'dataset_path'


In [62]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    EarlyStoppingCallback,
    BitsAndBytesConfig
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
from tqdm.auto import tqdm
from datetime import datetime
import torch
import torch.nn.functional as F
import os
import numpy as np

In [22]:
if __name__ == "__main__":
    image_embeddings_path = "/content/drive/MyDrive/LLM/Copy of clip_embeddings_instruct150k_15Oct_2024.npz"
    dataset_path = "/content/drive/MyDrive/Cap_project/llava_instruct_150k.json"
    output_dir = "/content/drive/MyDrive/LLM_training_19November"
    whisper_model_name = "openai/whisper-small"
    cache_dir = "/content/"
    dataset_dict, tokenizer = prepare_dataset(image_embeddings_path, dataset_path, cache_dir)
    # if os.path.exists(cache_dir):
    #     dataset_dict = DatasetDict.load_from_disk(cache_dir)
    # else:
    #   dataset_dict, tokenizer = prepare_dataset(image_embeddings_path, dataset_path, cache_dir)


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Map:   0%|          | 0/157712 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/289128 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36142 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36141 [00:00<?, ? examples/s]

In [63]:
# model_path = "/content/drive/MyDrive/LLM_training_17November"
# results = continue_training(
#     model_path=model_path,
#     train_dataset=dataset_dict['train'],
#     eval_dataset=dataset_dict['validation'],
#     output_dir="/content/drive/MyDrive/LLM_training_17November"
# )

# Usage
try:
    model_path = "/content/drive/MyDrive/LLM_training_20November/final_model"
    output_dir = "/content/drive/MyDrive/LLM_training_continued"

    results = continue_training(
        model_path=model_path,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['validation'],
        output_dir=output_dir
    )

except Exception as e:
    print(f"\nTraining failed: {str(e)}")
    import traceback
    traceback.print_exc()


Starting training at 2024-11-23 14:58:01
Model path: /content/drive/MyDrive/LLM_training_20November/final_model
Output directory: /content/drive/MyDrive/LLM_training_continued

Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Training failed: 'Phi3WithMultimodal' object has no attribute 'prepare_inputs_for_generation'


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/model.py", line 360, in __getattr__
    return super().__getattr__(name)  # defer to nn.Module's logic
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1931, in __getattr__
    raise AttributeError(
AttributeError: 'LoraModel' object has no attribute 'prepare_inputs_for_generation'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-63-350a63d48495>", line 14, in <cell line: 10>
    results = continue_training(
  File "<ipython-input-61-98c8e0a59643>", line 79, in continue_training
    model = get_peft_model(model, lora_config)
  File "/usr/local/lib/python3.10/dist-packages/peft/mapping.py", line 193, in get_peft_model
    return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1610, in __init__
   

In [ ]:
dataset_dict['test']


Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'image_embeddings'],
    num_rows: 36142
})

In [27]:
# Load the stored model
model_path = "/content/drive/MyDrive/LLM_training_output_1Nov2024_V3/final_model"  # Update with your model path
model, tokenizer = load_fine_tuned_models(model_path)

# Check accuracy on validation set
validation_accuracy = check_accuracy(model, tokenizer, dataset_dict['test'].select(range(2000))
)
print(f"Validation Accuracy: {validation_accuracy:.4f}")


config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


KeyboardInterrupt: 

In [ ]:
model


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): lora.Linear(
            (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPE

In [ ]:

# Fine-tune the model from the stored version
output_dir = "/content/drive/MyDrive/LLM_training_finetuned"  # Update with your desired output directory
modal, trainer = train_model(dataset_dict, tokenizer, output_dir)

In [ ]:
output_dir = "/content/drive/MyDrive/LLM_training_19Nov_test"
modal, trainer = train_model(dataset_dict, tokenizer, output_dir)

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/a90b62ae09941edff87a90ced39ba5807e6b2ade/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/a90b62ae09941edff87a90ced39ba5807e6b2ade/model.safetensors.index.json


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Saving model checkpoint to /content/drive/MyDrive/LLM_training_output_1Nov2024_V3/checkpoint-201
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/a90b62ae09941edff87a90ced39ba5807e6b2ade/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_

OOM error encountered. Attempting to save current state...


Saving model checkpoint to /content/drive/MyDrive/LLM_training_19Nov_test/emergency_checkpoint
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/a90b62ae09941edff87a90ced39ba5807e6b2ade/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_po

Emergency checkpoint saved to /content/drive/MyDrive/LLM_training_19Nov_test/emergency_checkpoint


OutOfMemoryError: CUDA out of memory. Tried to allocate 518.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 163.06 MiB is free. Process 45524 has 14.59 GiB memory in use. Of the allocated memory 13.61 GiB is allocated by PyTorch, and 862.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.evaluate()

NameError: name 'trainer' is not defined

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gc
import os

def load_and_save_model_from_checkpoint(checkpoint_path, target_save_path, device='cuda', load_in_8bit=False):
    """
    Load model from checkpoint and save it with memory optimization techniques.

    Args:
        checkpoint_path: Path to the checkpoint directory
        target_save_path: Where to save the final model
        device: 'cuda' or 'cpu'
        load_in_8bit: Whether to load the model in 8-bit precision
    """
    try:
        # Clear GPU memory before loading
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()

        # Load configuration separately first
        tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

        # Configure loading options
        load_options = {
            "torch_dtype": torch.float16 if device == 'cuda' else torch.float32,
            "low_cpu_mem_usage": True,
        }

        if load_in_8bit:
            load_options["load_in_8bit"] = True

        # Load model with memory optimizations
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint_path,
            **load_options
        )

        # Save model and tokenizer
        model.save_pretrained(target_save_path)
        tokenizer.save_pretrained(target_save_path)

        return True, "Model successfully loaded and saved"

    except Exception as e:
        return False, f"Error: {str(e)}"

def load_model_in_chunks(checkpoint_path, target_save_path, max_memory=None):
    """
    Load and save model using device map for very large models.

    Args:
        checkpoint_path: Path to the checkpoint directory
        target_save_path: Where to save the final model
        max_memory: Dict specifying maximum memory for each device
    """
    if max_memory is None:
        max_memory = {
            0: "24GiB",  # Adjust based on your GPU memory
            "cpu": "64GiB"  # Adjust based on your RAM
        }

    try:
        # Load with device map
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint_path,
            device_map="auto",
            max_memory=max_memory,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )

        tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

        # Save model and tokenizer
        model.save_pretrained(target_save_path)
        tokenizer.save_pretrained(target_save_path)

        return True, "Model successfully loaded and saved using device map"

    except Exception as e:
        return False, f"Error: {str(e)}"

In [ ]:
checkpoint_path = "/content/drive/MyDrive/LLM_training_19Nov_test/emergency_checkpoint"
save_path = "/content/drive/MyDrive/LLM_training_20November/final_model"

success, message = load_and_save_model_from_checkpoint(
    checkpoint_path,
    save_path,
    load_in_8bit=True  # Use 8-bit precision to reduce memory usage
)

TypeError: load_and_save_model_from_checkpoint() got an unexpected keyword argument 'load_in_4bit'

In [ ]:
success

True

In [ ]:
message

'Model successfully loaded and saved'

In [ ]:
# output_dir = "/content/drive/MyDrive/LLM_training_4hours_16 November"
# train_model(dataset_dict, tokenizer, output_dir)

In [ ]:
# !pip install --upgrade peft

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def load_model_for_inference(model_path, device='cuda'):
    """
    Load model and tokenizer for inference
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 if device == 'cuda' else torch.float32,
        device_map='auto'
    )
    model.eval()  # Set to evaluation mode
    return model, tokenizer

def generate_response(
    model,
    tokenizer,
    prompt,
    max_length=128,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    num_return_sequences=1,
    do_sample=True
):
    """
    Generate response from the model
    """
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_return_sequences=num_return_sequences,
        do_sample=do_sample,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decode and return responses
    responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return responses

# Example function for interactive chat
def chat_with_model(model, tokenizer):
    """
    Interactive chat loop with the model
    """
    print("Chat started (type 'quit' to exit)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'quit':
            break

        responses = generate_response(
            model,
            tokenizer,
            user_input,
            max_length=256  # Adjust based on your needs
        )

        print("\nModel:", responses[0])

In [ ]:
model_path = "/content/drive/MyDrive/LLM_training_17November/final_model"  # Adjust path
model, tokenizer = load_model_for_inference(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = "What is artificial intelligence?"
responses = generate_response(model, tokenizer, prompt)
print(responses[0])

What is artificial intelligence?

Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence. These tasks include problem-solving, learning, planning, understanding natural language, recognizing patterns, and making decisions. AI aims to create machines that can act and react intelligently, often mimicking human behavior.


### Instruction 2 (Much more difficult with at least 4 more constraints) Artificial intelligence (AI) encompasses the creation of computer systems capable of performing tasks that necessitate human-


In [ ]:
# from transformers import AutoProcessor, AutoModelForVisionTextDualEncoding
import torch
from PIL import Image

def setup_image_qa_model(model_path):
    """
    Set up model and processor for image question-answering
    """
    try:
        processor = AutoProcessor.from_pretrained(model_path)
        model = AutoModelForVisionTextDualEncoding.from_pretrained(
            model_path,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
        return model, processor
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return None, None

def ask_question_about_image(
    model,
    processor,
    image_path,
    question,
    max_length=128,
    temperature=0.7
):
    """
    Ask a question about an image and get a response
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path)

        # Prepare inputs
        inputs = processor(
            images=image,
            text=question,
            return_tensors="pt"
        )

        # Move inputs to the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate response
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True
        )

        # Decode response
        response = processor.decode(outputs[0], skip_special_tokens=True)
        return response

    except Exception as e:
        return f"Error processing image or generating response: {str(e)}"

def interactive_image_qa(model, processor, image_path):
    """
    Interactive Q&A session about an image
    """
    print(f"Image loaded from: {image_path}")
    print("Ask questions about the image (type 'quit' to exit)")

    while True:
        question = input("\nYour question: ")
        if question.lower() == 'quit':
            break

        response = ask_question_about_image(model, processor, image_path, question)
        print("\nResponse:", response)

In [ ]:
# !pip install --upgrade transformers

In [ ]:
# dataset_dict

***Training Finishes***



#Testing Date 1st Nov 2024

In [ ]:
!pip -q install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from evaluate import load
import json
from PIL import Image
from accelerate import Accelerator

# Load your fine-tuned model and required processors
def load_fine_tuned_models():
    # Quantize the model to 4-bit using bitsandbytes
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    # Load the model directly onto the device and enable gradient checkpointing
    model = AutoModelForCausalLM.from_pretrained(
        "/content/drive/MyDrive/LLM_training_output_1Nov2024_V3/final_model",
        quantization_config=bnb_config,
        trust_remote_code=True  # Required for loading custom models
    )
    model.gradient_checkpointing_enable()

    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
    tokenizer.pad_token = tokenizer.eos_token
    # Additional processor/loading functions here if needed
    return model, tokenizer

# Function to handle multimodal inference
def multimodal_inference(model, tokenizer, input_text=None, image_embedding=None, audio_embedding=None, device="cuda"):
    # Construct the input prompt with roles
    prompt_parts = ["You are a knowledgeable assistant."]
    if input_text:
        prompt_parts.append(f"User question: {input_text}")
    if image_embedding is not None:
        prompt_parts.append("An image is provided.")
    if audio_embedding is not None:
        prompt_parts.append("Audio information is provided.")

    # Combine all parts to form the final prompt
    prompt = " ".join(prompt_parts)

    # Tokenize the prompt for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Pass embeddings as model inputs if available
    inputs_embeds = {}
    if image_embedding is not None:
        inputs_embeds["image_embedding"] = image_embedding.unsqueeze(0).to(device)
    if audio_embedding is not None:
        inputs_embeds["audio_embedding"] = audio_embedding.unsqueeze(0).to(device)

    # Generate response from the model
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            **inputs_embeds  # add embeddings to the generate function call
        )

    # Decode and clean up the generated response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response

# Function to evaluate a single test example
def evaluate_example(example, model, tokenizer, device="cuda"):
    # Get inputs from the test example
    input_text = example.get("text")
    image_embedding = example.get("image_embedding")
    audio_embedding = example.get("audio_embedding")
    expected_response = example.get("expected_response")

    # Run inference
    generated_response = multimodal_inference(model, tokenizer, input_text, image_embedding, audio_embedding, device)

    # Print results
    print("Generated Response:", generated_response)
    print("Expected Response:", expected_response)

    # Optionally return evaluation metrics if there is an expected response
    return {"generated_response": generated_response, "expected_response": expected_response}

# Full evaluation over a test set
def evaluate_test_set(test_set_path, model, tokenizer, device="cuda"):
    with open(test_set_path, 'r') as f:
        test_set = json.load(f)

    results = []
    for example in test_set:
        result = evaluate_example(example, model, tokenizer, device)
        results.append(result)

    return results


In [ ]:
dataset_dict['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'image_embeddings'],
    num_rows: 36142
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json

# Load your fine-tuned model and required processors
def load_fine_tuned_models():
    # Quantize the model to 4-bit using bitsandbytes
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    # Load the model directly onto the device and enable gradient checkpointing
    model = AutoModelForCausalLM.from_pretrained(
        "/content/drive/MyDrive/LLM_training_output_1Nov2024_V3/final_model",
        quantization_config=bnb_config,
        trust_remote_code=True  # Required for loading custom models
    )
    model.gradient_checkpointing_enable()

    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Function to handle multimodal inference
def multimodal_inference(model, tokenizer, input_text=None, image_embedding=None, audio_embedding=None, device="cuda"):
    # Construct the input prompt with roles
    prompt_parts = ["You are a knowledgeable assistant."]
    if input_text:
        prompt_parts.append(f"User question: {input_text}")
    if image_embedding is not None:
        prompt_parts.append("An image is provided.")
    if audio_embedding is not None:
        prompt_parts.append("Audio information is provided.")

    # Combine all parts to form the final prompt
    prompt = " ".join(prompt_parts)

    # Tokenize the prompt for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Handle embedding inputs
    if image_embedding is not None:
        image_embedding = image_embedding.unsqueeze(0).to(device)
    if audio_embedding is not None:
        audio_embedding = audio_embedding.unsqueeze(0).to(device)

    # Generate response from the model with custom embeddings
    with torch.no_grad():
        outputs = model.generate(
            #input_ids=inputs["input_ids"],
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # Decode and clean up the generated response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response

# Function to evaluate a single test example
def evaluate_example(example, model, tokenizer, device="cuda"):
    # Get inputs from the test example
    input_text = example.get("text")
    image_embedding = example.get("image_embedding")
    audio_embedding = example.get("audio_embedding")
    expected_response = example.get("expected_response")

    # Run inference
    generated_response = multimodal_inference(model, tokenizer, input_text, image_embedding, audio_embedding, device)

    # Print results
    print("Generated Response:", generated_response)
    print("Expected Response:", expected_response)

    # Optionally return evaluation metrics if there is an expected response
    return {"generated_response": generated_response, "expected_response": expected_response}

# Full evaluation over a test set
def evaluate_test_set(test_set_path, model, tokenizer, device="cuda"):
    with open(test_set_path, 'r') as f:
        test_set = json.load(f)

    results = []
    for example in test_set:
        try:
            result = evaluate_example(example, model, tokenizer, device)
            results.append(result)
        except Exception as e:
            print(f"Error processing example {example}: {e}")
            continue  # Skip any problematic examples

    return results


In [ ]:
# Step 1: Load the fine-tuned model and tokenizer
model, tokenizer = load_fine_tuned_models()

# Step 2: Define the evaluation function over the Dataset object
def evaluate_test_set_hf(test_dataset, model, tokenizer, device="cuda"):
    results = []
    for example in test_dataset:
        # Prepare inputs for evaluation
        input_text = example.get("input_ids")  # Adapt as needed for your input text format
        image_embedding = example.get("image_embeddings")
        expected_response = example.get("labels")  # Assuming labels hold expected response text or IDs

        # Convert input_text and labels to text using tokenizer if they are token IDs
        if input_text is not None:
            input_text = tokenizer.decode(input_text, skip_special_tokens=True)
        if expected_response is not None:
            expected_response = tokenizer.decode(expected_response, skip_special_tokens=True)

        # Run evaluation
        result = evaluate_example(
            {"text": input_text, "image_embedding": image_embedding, "expected_response": expected_response},
            model,
            tokenizer,
            device
        )
        results.append(result)

    return results

# Step 3: Run evaluation on dataset_dict['test']
test_results = evaluate_test_set_hf(dataset_dict['test'], model, tokenizer, device="cuda")

# Print the results
for result in test_results:
    print("Generated Response:", result["generated_response"])
    print("Expected Response:", result["expected_response"])


configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Generated Response: You are a knowledgeable assistant. User question: 
 You are a helpful assistant.<|endofsystem|> What is the color of the thermometer in the image? The color of the thermometer in the image is white and red.      An image is provided. The image contains a thermometer, a mug, and a notebook. The thermometer has a white body and a red mercury column. The mug is a ceramic cup with a blue stripe and a handle. The notebook has a black cover and is open with a yellow pen on it. What color is the thermometer'rád akkor, hogyan érthetőek?

The thermometer in the image is white with a red
Expected Response: 
 You are a helpful assistant.<|endofsystem|> What is the color of the thermometer in the image? The color of the thermometer in the image is white and red.     
Generated Response: You are a knowledgeable assistant. User question: 
 You are a helpful assistant.<|endofsystem|> Are the stuffed bears in the image wearing various outfits? Yes, many of the stuffed bears are wea

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-cf8612b29d91>", line 31, in <cell line: 31>
    test_results = evaluate_test_set_hf(dataset_dict['test'], model, tokenizer, device="cuda")
  File "<ipython-input-36-cf8612b29d91>", line 20, in evaluate_test_set_hf
    result = evaluate_example(
  File "<ipython-input-35-402e26cdfefd>", line 73, in evaluate_example
    generated_response = multimodal_inference(model, tokenizer, input_text, image_embedding, audio_embedding, device)
  File "<ipython-input-35-402e26cdfefd>", line 51, in multimodal_inference
    outputs = model.generate(
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 2215, in gen

TypeError: object of type 'NoneType' has no len()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json

# Load your fine-tuned model and required processors
def load_fine_tuned_models():
    # Quantize the model to 4-bit using bitsandbytes
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    # Load the model directly onto the device and enable gradient checkpointing
    model = AutoModelForCausalLM.from_pretrained(
        "/content/drive/MyDrive/LLM_training_output_1Nov2024",
        quantization_config=bnb_config,
        trust_remote_code=True  # Required for loading custom models
    )
    model.gradient_checkpointing_enable()

    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Function to handle multimodal inference
def multimodal_inference(model, tokenizer, input_text=None, image_embedding=None, audio_embedding=None, device="cuda"):
    # Construct the input prompt with roles
    prompt_parts = ["You are a knowledgeable assistant."]
    if input_text:
        prompt_parts.append(f"User question: {input_text}")
    if image_embedding is not None:
        prompt_parts.append("An image is provided.")
    if audio_embedding is not None:
        prompt_parts.append("Audio information is provided.")

    # Combine all parts to form the final prompt
    prompt = " ".join(prompt_parts)

    # Tokenize the prompt for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Handle embedding inputs
    if image_embedding is not None:
        image_embedding = image_embedding.unsqueeze(0).to(device)
    if audio_embedding is not None:
        audio_embedding = audio_embedding.unsqueeze(0).to(device)

    # Generate response from the model with custom embeddings
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # Decode and clean up the generated response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response

# Function to evaluate a single test example
def evaluate_example(example, model, tokenizer, device="cuda"):
    # Get inputs from the test example
    input_text = example.get("text")
    image_embedding = example.get("image_embedding")
    audio_embedding = example.get("audio_embedding")
    expected_response = example.get("expected_response")

    # Run inference
    generated_response = multimodal_inference(model, tokenizer, input_text, image_embedding, audio_embedding, device)

    # Print results
    print("Generated Response:", generated_response)
    print("Expected Response:", expected_response)

    # Optionally return evaluation metrics if there is an expected response
    return {"generated_response": generated_response, "expected_response": expected_response}

# Full evaluation over a test set
def evaluate_test_set(test_set_path, model, tokenizer, device="cuda"):
    with open(test_set_path, 'r') as f:
        test_set = json.load(f)

    results = []
    for example in test_set:
        try:
            result = evaluate_example(example, model, tokenizer, device)
            results.append(result)
        except Exception as e:
            print(f"Error processing example {example}: {e}")
            continue  # Skip any problematic examples

    return results

In [ ]:
# Step 1: Load the fine-tuned model and tokenizer
model, tokenizer = load_fine_tuned_models()

# Step 2: Define the evaluation function over the Dataset object
def evaluate_test_set_hf(test_dataset, model, tokenizer, device="cuda"):
    results = []
    for example in test_dataset:
        # Prepare inputs for evaluation
        input_text = example.get("input_ids")  # Adapt as needed for your input text format
        image_embedding = example.get("image_embeddings")
        expected_response = example.get("labels")  # Assuming labels hold expected response text or IDs

        # Convert input_text and labels to text using tokenizer if they are token IDs
        if input_text is not None:
            input_text = tokenizer.decode(input_text, skip_special_tokens=True)
        if expected_response is not None:
            expected_response = tokenizer.decode(expected_response, skip_special_tokens=True)

        # Run evaluation
        result = evaluate_example(
            {"text": input_text, "image_embedding": image_embedding, "expected_response": expected_response},
            model,
            tokenizer,
            device
        )
        results.append(result)

    return results

# Step 3: Run evaluation on dataset_dict['test']
test_results = evaluate_test_set_hf(dataset_dict['test'], model, tokenizer, device="cuda")

# Print the results
for result in test_results:
    print("Generated Response:", result["generated_response"])
    print("Expected Response:", result["expected_response"])

# Inference starts

In [ ]:
output_dir = "/content/drive/MyDrive/LLM_training_4hours_16 November/final_model"


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_models(output_dir):
    """
    Load all required models and processors, including your fine-tuned model
    """
    try:
        # Load your fine-tuned model and tokenizer directly
        model = AutoModelForCausalLM.from_pretrained(
            output_dir,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(output_dir)

        # Load CLIP models
        clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

        # Load Whisper models
        whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

        # Set model to eval mode
        model.eval()
        torch.set_grad_enabled(False)

        return tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model

    except Exception as e:
        raise Exception(f"Error loading models: {str(e)}")


In [ ]:
from PIL import Image
import gc
import torch

def generate_response(
    model,
    tokenizer,
    clip_processor,
    clip_model,
    whisper_processor,
    whisper_model,
    input_image=None,
    input_text=None,
    input_audio=None,
    max_length=512,
    max_new_tokens=100,
    device='cuda'  # Set your device here
):
    """
    Generates a response from the model based on image, text, or audio input.
    """
    try:
        combined_prompt = "You are a knowledgeable assistant. "  # Initial prompt to set context

        # Process image input if provided
        if input_image is not None:
            if isinstance(input_image, str):
                image = Image.open(input_image).convert("RGB")  # Ensure image is in the correct format
            else:
                image = input_image

            image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                image_features = clip_model.get_image_features(**image_inputs)

            # Add a contextual prompt for the image
            combined_prompt += f"Context: An image is provided. {image_features}Describe relevant features or subjects in the image. "

        # Process text input if provided
        if input_text is not None:
            combined_prompt += f"User Question: {input_text} "

        # Process audio input if provided
        if input_audio is not None:
            audio_inputs = whisper_processor(input_audio, return_tensors="pt").input_features.to(device)
            with torch.no_grad():
                audio_outputs = whisper_model.generate(audio_inputs)
                transcription = whisper_processor.decode(audio_outputs[0])

            combined_prompt += f"Transcription from audio: {transcription} "

        # Finalize the prompt
        combined_prompt += "Please provide a detailed answer based on the above context."

        print(combined_prompt)

        # Tokenize combined prompt with image context
        inputs = tokenizer(
            combined_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            padding=True
        ).to(device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                num_return_sequences=1,
            )

        # Decode and return the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Clean up memory
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()

        return response

    except Exception as e:
        raise Exception(f"Error in generate_response: {str(e)}")


In [ ]:
import torch
from torch.nn import functional as F
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel
from transformers import WhisperProcessor, WhisperForConditionalGeneration
# Import the 'clip' module
from transformers import CLIPTokenizer, CLIPTextModel # Import CLIPTokenizer and CLIPTextModel
from PIL import Image
import gc

In [ ]:
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc

import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc

def generate_response(
    model,
    tokenizer,
    clip_processor,
    clip_model,
    whisper_processor,
    whisper_model,
    input_image=None,
    input_text=None,
    input_audio=None,
    max_length=512,
    max_new_tokens=100,
    device='cuda'
):
    """
    Generates a response from the model based on image, text, or audio input.
    """
    try:
        # Initialize prompt components
        prompt_parts = ["You are a knowledgeable assistant."]

        # Process image input if provided
        if input_image is not None:
            if isinstance(input_image, str):
                image = Image.open(input_image).convert("RGB")
            else:
                image = input_image

            # Generate image features with CLIP
            image_features = generate_image_features(clip_model, clip_processor, image, device)


            prompt_parts.append(f"Context: The following image features are provided: [{image_features}...]")

        # Process text input if provided
        if input_text is not None:
            prompt_parts.append(f"User Question: {input_text}")

        # Process audio input if provided
        if input_audio is not None:
            audio_inputs = whisper_processor(input_audio, return_tensors="pt").input_features.to(device)
            with torch.no_grad():
                audio_outputs = whisper_model.generate(audio_inputs)
                transcription = whisper_processor.decode(audio_outputs[0])
            prompt_parts.append(f"Transcription from audio: {transcription}")

        # Combine all parts with proper spacing
        prompt_parts.append("Please provide a detailed answer based on the above context.")
        combined_prompt = " ".join(prompt_parts)
        print(combined_prompt)
        # Tokenize the prompt
        inputs = tokenizer(
            combined_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            padding=True,
            add_special_tokens=True
        ).to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2,
                num_return_sequences=1,
            )

        # Decode and clean up response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.replace(combined_prompt, "").strip()

        # Memory cleanup
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()

        return response

    except Exception as e:
        raise Exception(f"Error in generate_response: {str(e)}")

def generate_image_features(clip_model, clip_processor, image, device):
    """
    Generates a compact feature vector of an image using CLIP.
    """
    # Process the image with CLIP to get feature vector
    image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**image_inputs)

        # Normalize and reduce features to a compact string representation
        image_features_normalized = F.normalize(image_features, p=2, dim=1)
        features_str = ",".join([f"{val:.2f}" for val in image_features_normalized[0][:10].cpu().numpy()])

    return features_str


In [ ]:
output_dir = "/content/drive/MyDrive/LLM_training_4hours_16 November/final_model"
#output_dir = "/content/drive/MyDrive/LLM_training_output_1Nov2024"
#Load models first
tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model = load_models(output_dir)

In [ ]:
# model

In [ ]:
generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/test.jpg", input_text="describe the image?")

In [ ]:
# image_features = clip_model.get_image_features(**input_image)

In [ ]:
input_image